In [1]:
import numpy as np
import pandas as pd
from scipy import stats

from matplotlib import pyplot as plt
import seaborn as sns
% matplotlib inline

# Plan of attack

* Find an investment property based on 2015 data: Jan 1 - dec 31
* "Invest" as of Jan 1 2016 and see if our investment held well

### Process
* Things to look at: start with price to rent as of dec 2015
* For the top 50 (lowest ptr) Find which ones had their rent grow the most from Jan 1 - dec 31
* Then compare growth of sales price
* Do this for one bed and two beds

In [2]:
price_to_rent = pd.read_csv('./data/price-to-rent-ratio.csv')
price_to_rent.head(1)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2010-10,2010-11,2010-12,...,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02
0,274772,Northeast Dallas,Dallas,TX,Dallas-Fort Worth,Dallas,1,10.0,9.91,9.73,...,11.72,11.77,12.14,12.17,12.21,12.34,12.54,12.61,12.57,12.64


In [35]:
price_to_rent.shape

(5683, 84)

In [19]:
start_date = '2015-01'
end_date = '2015-12'

start_i = price_to_rent.columns.get_loc(start_date)
end_i = price_to_rent.columns.get_loc(end_date)

ptr_for_date = price_to_rent.iloc[:, [3,2,1,end_i]]

# ptr_for_date = price_to_rent.iloc[:,start_i:end_i]
# ptr_for_date["State"] = price_to_rent.iloc[:,3]
# ptr_for_date["City"] = price_to_rent.iloc[:,2]
# ptr_for_date["Region"] = price_to_rent.iloc[:,1]

sorted_ptr = ptr_for_date.sort_values("2015-12")

# QUESTION: How do we create a new column based on this data's index?
#sorted_ptr = sorted_ptr.reset_index()

sorted_ptr.head(10)

,index,State,City,RegionName,2015-12
0,2621,NY,East Hampton,Springs,2.37
1,4046,NY,Riverhead,Aquebogue,2.48
2,3105,MD,Baltimore,East Baltimore Midway,2.55
3,4494,MD,Baltimore,Biddle Street,2.66
4,3111,MD,Baltimore,Broadway East,2.74
5,5060,MD,Baltimore,Darley Park,2.81
6,4246,MD,Baltimore,Shipley Hill,2.82
7,5120,MD,Baltimore,Mosher,2.82
8,5346,MD,Baltimore,South Clifton Park,3.07
9,4723,MD,Baltimore,Winchester,3.07


In [39]:
# Find rent growth

rental_1_room = pd.read_csv("./data/rental-1-room.csv")
rental_1_room.head(1)

,RegionName,City,State,Metro,CountyName,SizeRank,2010-02,2010-03,2010-04,2010-05,...,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02
0,Northeast Dallas,Dallas,TX,Dallas-Fort Worth,Dallas,1,NaN,NaN,NaN,NaN,...,769.5,806.5,829.0,820.0,835.0,850.0,820.0,899.5,855.0,895.0


In [40]:
rental_1_room["2015_growth"] = \
    (rental_1_room[end_date] - rental_1_room[start_date]) / rental_1_room[start_date]

rental_1_sorted = rental_1_room.sort_values("2015_growth", ascending=False)
rental_1_sorted.head(1)

,RegionName,City,State,Metro,CountyName,SizeRank,2010-02,2010-03,2010-04,2010-05,...,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2015_growth
234,Victoria Park,Fort Lauderdale,FL,Miami-Fort Lauderdale,Broward,235,NaN,NaN,NaN,NaN,...,1400.0,1275.0,1350.0,1400.0,1450.0,1300.0,1275.0,1250.0,1350.0,0.3125


In [41]:
# https://cl.ly/1p0w0327122j

purchase_1_room = pd.read_csv("./data/purchase-data/purchase/Neighborhood_Zhvi_1bedroom.csv")
purchase_1_room.head(1)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02
0,270958,Upper West Side,New York,NY,New York,New York,1,NaN,NaN,NaN,...,862500.0,872800,884100,897000,909800,921100,929400,937500,951000,966800


In [43]:
purchase_1_room["2015_growth"] = \
    (purchase_1_room[end_date] - purchase_1_room[start_date]) / purchase_1_room[start_date]

purchase_1_room = purchase_1_room.sort_values("2015_growth", ascending=False)
rental_1_sorted.head()

,RegionName,City,State,Metro,CountyName,SizeRank,2010-02,2010-03,2010-04,2010-05,...,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2015_growth
234,Victoria Park,Fort Lauderdale,FL,Miami-Fort Lauderdale,Broward,235,NaN,NaN,NaN,NaN,...,1400.0,1275.0,1350.0,1400.0,1450.0,1300.0,1275.0,1250.0,1350.0,0.312500
59,Encanto,Phoenix,AZ,Phoenix,Maricopa,60,NaN,NaN,NaN,NaN,...,812.5,883.5,850.0,875.0,795.0,795.0,785.0,797.0,795.0,0.304348
208,Oceanfront,Miami Beach,FL,Miami-Fort Lauderdale,Miami-Dade,209,NaN,NaN,NaN,NaN,...,2400.0,2300.0,2200.0,2300.0,2200.0,2450.0,2200.0,2350.0,2300.0,0.250625
196,First Hill,Seattle,WA,Seattle,King,197,NaN,NaN,NaN,NaN,...,2030.0,1926.0,1970.0,1972.5,1842.5,1832.5,1800.0,1706.5,1775.0,0.234168
8,Deer Valley,Phoenix,AZ,Phoenix,Maricopa,9,NaN,NaN,NaN,NaN,...,787.0,826.0,915.0,812.0,877.0,867.0,850.0,801.0,852.0,0.234043
